# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nova sintra,14.8667,-24.7167,23.41,83,6,12.93,CV,1713612663
1,1,invercargill,-46.4000,168.3500,10.01,90,100,2.24,NZ,1713612663
2,2,okhotsk,59.3833,143.3000,-3.72,99,100,0.54,RU,1713612664
3,3,bilibino,68.0546,166.4372,-17.07,97,52,2.43,RU,1713612665
4,4,beliator,23.3333,87.2167,42.08,9,14,5.65,IN,1713612666


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display


# Configure the map plot
map_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=800,
    frame_height=600,
    color="City",
    size="Humidity"
)

# Display the map
%capture c --no-display 
map_1

C:\Users\User\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
df= city_data_df.loc[(city_data_df["Cloudiness"]<=0)
                                  &(city_data_df["Max Temp"]<30)
                                  &(city_data_df["Max Temp"]>21)
                                  &(city_data_df["Wind Speed"]<7)
                                  &(city_data_df["Humidity"]<75)]    



# Drop any rows with null values
df.dropna()

# Display sample data
df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,masvingo,-20.0637,30.8277,28.62,25,0,4.81,ZW,1713612710
123,123,carnarvon,-24.8667,113.6333,26.94,40,0,5.98,AU,1713612769
125,125,kailua-kona,19.6406,-155.9956,22.98,68,0,1.54,US,1713612771
289,289,grootfontein,-19.5667,18.1167,29.76,26,0,4.12,NaN,1713612910
404,404,minab,27.1467,57.0801,28.90,40,0,4.52,IR,1713613007
444,444,fort-shevchenko,44.5086,50.2630,21.20,42,0,2.65,KZ,1713613040


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
 
hotel_df=df[["City","Country","Lat","Lng","Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

#hotel_df["Hotel Name"]="-----"
hotel_df.insert(5, "Hotel Name", "------") 

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
53,masvingo,ZW,-20.0637,30.8277,25,------
123,carnarvon,AU,-24.8667,113.6333,40,------
125,kailua-kona,US,19.6406,-155.9956,68,------
289,grootfontein,NaN,-19.5667,18.1167,26,------
404,minab,IR,27.1467,57.0801,40,------
444,fort-shevchenko,KZ,44.5086,50.2630,42,------


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories="accommodation.hotel"
params = {"apiKey":geoapify_key,
          "categories":categories}
         

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    Latitude=row["Lat"]
    Longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL (provided)
    base_url = "https://api.geoapify.com/v2/places"
    


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
         # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
masvingo - nearest hotel: Hotel Victoria
carnarvon - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
grootfontein - nearest hotel: Meteor Travel Inn
minab - nearest hotel: هتل صدف
fort-shevchenko - nearest hotel: Akku


,City,Country,Lat,Lng,Humidity,Hotel Name
53,masvingo,ZW,-20.0637,30.8277,25,Hotel Victoria
123,carnarvon,AU,-24.8667,113.6333,40,No hotel found
125,kailua-kona,US,19.6406,-155.9956,68,Kona Seaside Hotel
289,grootfontein,NaN,-19.5667,18.1167,26,Meteor Travel Inn
404,minab,IR,27.1467,57.0801,40,هتل صدف
444,fort-shevchenko,KZ,44.5086,50.2630,42,Akku


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display


# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=800,
    frame_height=600,
    color="City",
    size="Humidity",
    hover_cols = ["City", "Country","Lat","Lng","Humidity","Hotel Name"]
    )

# Display the map
%capture c --no-display 
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)